# RandomForestRegressor

2017-05-22

In [21]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import pandas as pd
from datetime import datetime,timedelta,date,time

### 录入数据

In [22]:
travel_time_train_data = pd.read_csv('/home/godcedric/GitLocal/KDDCUP2017/加工过的数据集/3.0/travel_time_train_data.csv')
volume_train_data = pd.read_csv('/home/godcedric/GitLocal/KDDCUP2017/加工过的数据集/3.0/volume_train_data.csv')
travel_time_test_data = pd.read_csv('/home/godcedric/GitLocal/KDDCUP2017/加工过的数据集/3.0/test_travel_time_data.csv')
volume_test_data = pd.read_csv('/home/godcedric/GitLocal/KDDCUP2017/加工过的数据集/3.0/test_volume_data.csv')

### 准备数据

In [23]:
# 分割训练集和验证集
test_date = ['2016-10-11', '2016-10-12', '2016-10-13', '2016-10-14', '2016-10-15', '2016-10-16', '2016-10-17']
def isin(x):
    if x in test_date:
        return 1
    else:
        return 0
# 平均时间
travel_time_train_data['is_test'] = travel_time_train_data['date'].map(isin)
travel_time_train = travel_time_train_data[travel_time_train_data['is_test'] == 0]
travel_time_val = travel_time_train_data[travel_time_train_data['is_test'] == 1]
# 流量
volume_train_data['is_test'] = volume_train_data['date'].map(isin)
volume_train = volume_train_data[volume_train_data['is_test'] == 0]
volume_val = volume_train_data[volume_train_data['is_test'] == 1]

In [24]:
# 分离标签和特征
Y_time = travel_time_train.avg_travel_time
X_time = travel_time_train.drop(['avg_travel_time'], axis=1)
val_Y_time = travel_time_val.avg_travel_time
val_X_time = travel_time_val.drop(['avg_travel_time'], axis=1)


Y_volume = volume_train.volume
X_volume = volume_train.drop(['volume'], axis=1)
val_Y_volume = volume_val.volume
val_X_volume = volume_val.drop(['volume'], axis=1)

In [25]:
# 平均时间
# 选择特征
X_time = X_time.drop(['intersection_id', 'tollgate_id', 'time_window', 'start_time', 'date', 'time'], axis=1)
X_time = X_time.drop(['wind_direction2', 'wind_speed2', 'precipitation2', 'SSD_level'], axis=1)
X_time = X_time.drop(['last_40min', 'last_60min', 'last_80min', 'last_100min', 'last_120min'], axis=1)
X_time = X_time.drop(['datemap', 'is_test'], axis=1)
#X_time = X_time.drop(['last_20min'], axis=1)
X_time = X_time.drop(['route', 'timemap', 'hour', 'minute', 'weekday', 'is_workday'], axis=1)

val_X_time = val_X_time.drop(['intersection_id', 'tollgate_id', 'time_window', 'start_time', 'date', 'time'], axis=1)
val_X_time = val_X_time.drop(['wind_direction2', 'wind_speed2', 'precipitation2', 'SSD_level'], axis=1)
val_X_time = val_X_time.drop(['last_40min', 'last_60min', 'last_80min', 'last_100min', 'last_120min'], axis=1)
val_X_time = val_X_time.drop(['datemap', 'is_test'], axis=1)
#val_X_time = val_X_time.drop(['last_20min'], axis=1)
val_X_time = val_X_time.drop(['route', 'timemap', 'hour', 'minute', 'weekday', 'is_workday'], axis=1)

# 测试集同步
travel_time_submission = travel_time_test_data[['intersection_id', 'tollgate_id', 'time_window']]
travel_time_test_data = travel_time_test_data.drop(['intersection_id', 'tollgate_id', 'time_window', 'start_time', 'date', 'time'], axis=1)
travel_time_test_data = travel_time_test_data.drop(['wind_direction2', 'wind_speed2', 'precipitation2', 'SSD_level'], axis=1)
travel_time_test_data = travel_time_test_data.drop(['last_40min', 'last_60min', 'last_80min', 'last_100min', 'last_120min'], axis=1)
travel_time_test_data = travel_time_test_data.drop(['datemap'], axis=1)
#travel_time_test_data = travel_time_test_data.drop(['last_20min'], axis=1)
travel_time_test_data = travel_time_test_data.drop(['route', 'timemap', 'hour', 'minute', 'weekday', 'is_workday'], axis=1)

In [26]:
# 流量
# 选择特征
X_volume = X_volume.drop(['tollgate_id', 'direction', 'time_window', 'start_time', 'date', 'time'], axis=1)
X_volume = X_volume.drop(['wind_direction2', 'wind_speed2', 'precipitation2', 'SSD_level'], axis=1)
X_volume = X_volume.drop(['last_40min', 'last_60min', 'last_80min', 'last_100min', 'last_120min'], axis=1)
X_volume = X_volume.drop(['datemap', 'is_test'], axis=1)
#X_volume = X_volume.drop(['last_20min'], axis=1)
X_volume = X_volume.drop(['pair', 'timemap', 'hour', 'minute', 'weekday', 'is_workday'], axis=1)

val_X_volume = val_X_volume.drop(['tollgate_id', 'direction', 'time_window', 'start_time', 'date', 'time'], axis=1)
val_X_volume = val_X_volume.drop(['wind_direction2', 'wind_speed2', 'precipitation2', 'SSD_level'], axis=1)
val_X_volume = val_X_volume.drop(['last_40min', 'last_60min', 'last_80min', 'last_100min', 'last_120min'], axis=1)
val_X_volume = val_X_volume.drop(['datemap', 'is_test'], axis=1)
#val_X_volume = val_X_volume.drop(['last_20min'], axis=1)
val_X_volume = val_X_volume.drop(['pair', 'timemap', 'hour', 'minute', 'weekday', 'is_workday'], axis=1)

# 测试集同步
volume_submission = volume_test_data[['tollgate_id', 'time_window', 'direction']]
volume_test_data = volume_test_data.drop(['tollgate_id', 'direction', 'time_window', 'start_time', 'date', 'time'], axis=1)
volume_test_data = volume_test_data.drop(['wind_direction2', 'wind_speed2', 'precipitation2', 'SSD_level'], axis=1)
volume_test_data = volume_test_data.drop(['last_40min', 'last_60min', 'last_80min', 'last_100min', 'last_120min'], axis=1)
volume_test_data = volume_test_data.drop(['datemap'], axis=1)
#volume_test_data = volume_test_data.drop(['last_20min'], axis=1)
volume_test_data = volume_test_data.drop(['pair', 'timemap', 'hour', 'minute', 'weekday', 'is_workday'], axis=1)

In [27]:
X_time.columns.values

array(['pressure', 'wind_direction', 'wind_speed', 'temperature',
       'rel_humidity', 'precipitation', 'last_20min', 'SSD',
       'in_link_cross_conut', 'out_link_cross_count', 'length',
       'link_count', '1_length', '2_length', '3_length', '4_length',
       '1_count', '2_count', '3_count', '4_count', 'A-2', 'A-3', 'B-1',
       'B-3', 'C-1', 'C-3', 'hour__0', 'hour__1', 'hour__2', 'hour__3',
       'hour__4', 'hour__5', 'hour__6', 'hour__7', 'hour__8', 'hour__9',
       'hour__10', 'hour__11', 'hour__12', 'hour__13', 'hour__14',
       'hour__15', 'hour__16', 'hour__17', 'hour__18', 'hour__19',
       'hour__20', 'hour__21', 'hour__22', 'hour__23', 'minute__0',
       'minute__20', 'minute__40', 'weekday__0', 'weekday__1',
       'weekday__2', 'weekday__3', 'weekday__4', 'weekday__5',
       'weekday__6', 'workday__1', 'workday__2', 'workday__3'], dtype=object)

In [8]:
# 字符串特征转换数值
X_time['route'] = pd.factorize(X_time['route'])[0]
val_X_time['route'] = pd.factorize(val_X_time['route'])[0]
travel_time_test_data['route'] = pd.factorize(travel_time_test_data['route'])[0]
X_volume['pair'] = pd.factorize(X_volume['pair'])[0]
val_X_volume['pair'] = pd.factorize(val_X_volume['pair'])[0]
volume_test_data['pair'] = pd.factorize(volume_test_data['pair'])[0]

KeyError: 'route'

### 训练

In [28]:
# 自定义评价函数
def MAPE(preds, realval):
    return 'MAPE', float(sum(np.fabs((realval - preds) / realval))) / len(realval)

In [29]:
# 默认参数
rf_default = RandomForestRegressor(oob_score = True, random_state = 10)
rf_default.fit(X_time, Y_time)
print(rf_default.oob_score_)  

# 验证集情况
preds = rf_default.predict(val_X_time)

print(MAPE(preds, val_Y_time))

# 预测及输出


0.530465752499
('MAPE', 0.19799182788658606)


/home/godcedric/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:723: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "


In [30]:
X_volume.columns.values

array(['pressure', 'wind_direction', 'wind_speed', 'temperature',
       'rel_humidity', 'precipitation', 'last_20min', 'SSD', '1-0', '1-1',
       '2-0', '3-0', '3-1', 'hour__0', 'hour__1', 'hour__2', 'hour__3',
       'hour__4', 'hour__5', 'hour__6', 'hour__7', 'hour__8', 'hour__9',
       'hour__10', 'hour__11', 'hour__12', 'hour__13', 'hour__14',
       'hour__15', 'hour__16', 'hour__17', 'hour__18', 'hour__19',
       'hour__20', 'hour__21', 'hour__22', 'hour__23', 'minute__0',
       'minute__20', 'minute__40', 'weekday__0', 'weekday__1',
       'weekday__2', 'weekday__3', 'weekday__4', 'weekday__5',
       'weekday__6', 'workday__1', 'workday__2', 'workday__3'], dtype=object)

In [31]:
rf_default2 = RandomForestRegressor(oob_score = True, random_state = 10)
rf_default2.fit(X_volume, Y_volume)
print(rf_default2.oob_score_) 

# 验证集情况
preds2 = rf_default2.predict(val_X_volume)

print(MAPE(preds2, val_Y_volume))

0.878007502261
('MAPE', 0.27008234326181885)


/home/godcedric/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:723: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
